In [ ]:
%matplotlib inline
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from numpy import pi

# 기능적 세포 정의 (이제 이름 지정 없이 구성)
def functional_cell():
    cell = QuantumCircuit(2)
    cell.h(0)
    cell.cx(0, 1)
    cell.ry(pi / 4, 1)
    cell.cx(0, 1)
    return cell

# 회로 생성 함수 (to_instruction 사용하지 않음)
def build_repeated_circuit(repeat_count):
    qc = QuantumCircuit(2)
    for _ in range(repeat_count):
        cell = functional_cell()
        qc.compose(cell, inplace=True)
    qc.measure_all()
    return qc

# 실험 회로 구성
circuits = {
    "Repeat 1x": build_repeated_circuit(1),
    "Repeat 2x": build_repeated_circuit(2),
    "Repeat 3x": build_repeated_circuit(3),
}

# 시뮬레이터 실행
simulator = AerSimulator()
results = {}

for label, circuit in circuits.items():
    job = simulator.run(circuit, shots=1024)
    result = job.result()
    counts = result.get_counts()
    results[label] = counts

# 결과 시각화
for label, counts in results.items():
    plot_histogram(counts, title=f"Result: {label}")
    plt.show()

for label, counts in results.items():
    plot_histogram(counts, title=f"Result: {label}")
    plt.savefig(f"{label.replace(' ', '_')}.png")
    plt.show()

In [ ]:
#정량 지표 분석 코드 삽입 :

%matplotlib inline
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from numpy import pi
import numpy as np
from scipy.spatial.distance import cosine

# 기능적 세포 정의 (이제 이름 지정 없이 구성)
def functional_cell():
    cell = QuantumCircuit(2)
    cell.h(0)
    cell.cx(0, 1)
    cell.ry(pi / 4, 1)
    cell.cx(0, 1)
    return cell

# 회로 생성 함수 (to_instruction 사용하지 않음)
def build_repeated_circuit(repeat_count):
    qc = QuantumCircuit(2)
    for _ in range(repeat_count):
        cell = functional_cell()
        qc.compose(cell, inplace=True)
    qc.measure_all()
    return qc

# 정량 지표 함수 정의
def shannon_entropy(counts):
    total = sum(counts.values())
    probs = [v / total for v in counts.values()]
    return -sum(p * np.log2(p) for p in probs if p > 0)

def distribution_vector(counts, keys):
    total = sum(counts.values())
    return np.array([counts.get(k, 0) / total for k in keys])

# 실험 회로 구성
circuits = {
    "Repeat 1x": build_repeated_circuit(1),
    "Repeat 2x": build_repeated_circuit(2),
    "Repeat 3x": build_repeated_circuit(3),
}

# 시뮬레이터 실행
simulator = AerSimulator()
results = {}

for label, circuit in circuits.items():
    job = simulator.run(circuit, shots=1024)
    result = job.result()
    counts = result.get_counts()
    results[label] = counts

# 결과 시각화
for label, counts in results.items():
    plot_histogram(counts, title=f"Result: {label}")
    plt.show()

    # 결과 저장
    plt.savefig(f"{label.replace(' ', '_')}.png")
    plt.close()

# 정량 지표 분석
labels = list(results.keys())
entropy_values = [shannon_entropy(results[label]) for label in labels]

print("📊 Shannon Entropy per Repeat:")
for label, entropy in zip(labels, entropy_values):
    print(f"{label}: {entropy:.4f}")

print("\n📊 Cosine Similarity Between Repeats:")
for i in range(len(labels) - 1):
    c1, c2 = results[labels[i]], results[labels[i + 1]]
    all_keys = sorted(set(c1) | set(c2))
    v1 = distribution_vector(c1, all_keys)
    v2 = distribution_vector(c2, all_keys)
    sim = 1 - cosine(v1, v2)
    print(f"{labels[i]} vs {labels[i+1]}: {sim:.4f}")

# 엔트로피 시각화
plt.plot(range(1, len(entropy_values)+1), entropy_values, marker='o')
plt.xticks(ticks=range(1, len(entropy_values)+1), labels=labels)
plt.title("Shannon Entropy Across Feedback Repeats")
plt.xlabel("Repeat")
plt.ylabel("Entropy")
plt.grid(True)
plt.show()
